Import packages

In [23]:
import tensorflow as tf
import create_dataset
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Define generic functions to initialize weight/bias variables, and to define common convolution and pooling operations

In [24]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name='weights')

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial, name='biases')

In [25]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

Define the network architecture + loss function and metrics

In [43]:
# Input/Output placeholders
x = tf.placeholder(tf.float32, shape=[None, 121], name='input')
y_ = tf.placeholder(tf.float32, shape=[None, 2], name='output')

num_fc_layers = 10
num_nodes = 121

layers = [x]
keep_prob = tf.placeholder(tf.float32)
prev_layer_size = 121                     
# Create FC layers
for i in range(0, num_fc_layers):
    with tf.name_scope('{}/FC'.format(i)):
        W_fc = weight_variable([prev_layer_size, num_nodes])
        b_fc = bias_variable([121])
        h_fc = tf.nn.relu(tf.matmul(layers[-1], W_fc) + b_fc)
        print layers[-1]
        h_fc_drop = tf.nn.dropout(h_fc, keep_prob)
        layers.append(h_fc_drop)
        prev_layer_size = num_nodes
        

# Readout
with tf.name_scope('5/Readout'):
    W_fc2 = weight_variable([num_nodes, 2])
    b_fc2 = bias_variable([2])
    y_conv = tf.matmul(layers[-1], W_fc2) + b_fc2
    norm_y_conv = tf.nn.l2_normalize(y_conv, dim=0)

# Define loss
l2_loss = tf.reduce_mean(
    tf.nn.l2_loss(tf.subtract(norm_y_conv, y_)))
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(l2_loss)

Tensor("input_18:0", shape=(?, 121), dtype=float32)
Tensor("0/FC_18/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("1/FC_18/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("2/FC_18/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("3/FC_18/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("4/FC_18/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("5/FC_18/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("6/FC_18/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("7/FC_18/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("8/FC_18/dropout/mul:0", shape=(?, 121), dtype=float32)


Train the network per-batch

In [45]:
batch_size = 120
logging_frequency = 50
iterations = 2000

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(iterations):
    batch = create_dataset.generate_data_angles(batch_size)
    if i % logging_frequency == 0:
      loss = sess.run(l2_loss, feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0}) 
      print('step %d, training loss %g' % (i, loss/batch_size))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})

step 0, training loss 0.503562
step 50, training loss 0.502117
step 100, training loss 0.503753
step 150, training loss 0.503433
step 200, training loss 0.502871
step 250, training loss 0.493297
step 300, training loss 0.492976
step 350, training loss 0.496417
step 400, training loss 0.497018
step 450, training loss 0.454415
step 500, training loss 0.453868
step 550, training loss 0.399878
step 600, training loss 0.407253
step 650, training loss 0.394383
step 700, training loss 0.379364
step 750, training loss 0.377259
step 800, training loss 0.376433
step 850, training loss 0.376162
step 900, training loss 0.376004
step 950, training loss 0.375897
step 1000, training loss 0.375832
step 1050, training loss 0.375787
step 1100, training loss 0.375757
step 1150, training loss 0.375734
step 1200, training loss 0.375718
step 1250, training loss 0.375705
step 1300, training loss 0.375694
step 1350, training loss 0.375686
step 1400, training loss 0.375678
step 1450, training loss 0.375671
ste